In [ ]:
def myfunc(a, b, c=0):
    "Return a-b if a>b, otherwise return a+b"
    if a > b:
        return np.array([a - b - c, a - b - c])
    else:
        return np.array([a + b + c, a - b - c])

In [ ]:
def myfunc(a, b=2):
    "Return a-b if a>b, otherwise return a+b"
    return a*5 + b

In [ ]:
a = np.ones(5)
myfunc(a)

array([7., 7., 7., 7., 7.])

In [ ]:
import numpy as np
vfunc = np.vectorize(myfunc, otypes=[np.ndarray])

ValueError: Invalid otype specification

In [ ]:
ret = vfunc([1, 2, 3, 4], 2, c=2)
ret

array([array([ 5, -3]), array([ 6, -2]), array([-1, -1]), array([0, 0])],
      dtype=object)

In [ ]:
seq = np.zeros((4, 2))
for i in range(4):
    seq[i] = ret[i]

In [ ]:
seq

array([[ 5., -3.],
       [ 6., -2.],
       [-1., -1.],
       [ 0.,  0.]])

In [ ]:
vfunc([1, 2, 3, 4], 2, c=2)
np.concatenate(vfunc([1, 2, 3, 4], 2, c=2), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
import torch

In [ ]:
torch.ones(4)

tensor([1., 1., 1., 1.])

In [ ]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
a = np.ones(10)
a

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
a[7:12]

array([1., 1., 1.])

In [ ]:
Na, Nq = 6, 3

In [ ]:
Na**Nq

216

In [ ]:
4//5

0

In [ ]:
-(7 // -5)

2

In [ ]:
import math

In [ ]:
math.ceil(7/5)

2

In [ ]:
def int2basestr(n, base, l=0):
    """Convert int n into a b-nary list of size l e.g. returns [0, 1, 0] for
    (2, 2, l=3)

    Args:
        n (int) : Integer to convert to a b-nary representation
        b (int) : Base of the b-nary representation e.g. 2 -> {0, 1},
                  3 -> {0, 1, 2}
        l (int) : Length of the b-nary list, if precision is not enough, 
                  it overflows

    Returns:
        (list) : List of size l? i.e. the b-nary representation of n
    """
    d = int(n % base)
    if d == n:
        return [0 for _ in range(l-1)] + [d]
    else:
        a = int2basestr(int((n-d)/base), base) + [d]
        return [0 for _ in range(l-len(a))] + a

def basestr2int(st, base):
    return sum([st[i] * base**(len(st)-i-1) for i in range(len(st))])

In [ ]:
Na, Nq = 4, 2
outcomes, probs = np.arange(Na**Nq), np.ones(Na**Nq)

In [ ]:
outcomes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [ ]:
v_int2basestr = np.vectorize(int2basestr, otypes=[np.ndarray])
batch_sequences = np.zeros((len(outcomes), Nq))
ret = v_int2basestr(outcomes, Na, l=Nq)
ret

array([list([0, 0]), list([0, 1]), list([0, 2]), list([0, 3]),
       list([1, 0]), list([1, 1]), list([1, 2]), list([1, 3]),
       list([2, 0]), list([2, 1]), list([2, 2]), list([2, 3]),
       list([3, 0]), list([3, 1]), list([3, 2]), list([3, 3])],
      dtype=object)

In [ ]:
for i in range(len(outcomes)):
    batch_sequences[i] = ret[i]

In [ ]:
torch.as_tensor(batch_sequences)

tensor([[0., 0.],
        [0., 1.],
        [0., 2.],
        [0., 3.],
        [1., 0.],
        [1., 1.],
        [1., 2.],
        [1., 3.],
        [2., 0.],
        [2., 1.],
        [2., 2.],
        [2., 3.],
        [3., 0.],
        [3., 1.],
        [3., 2.],
        [3., 3.]], dtype=torch.float64)

In [ ]:
def GenPOVMProbTable(batch_size):
    """Build POVM probability table using the generator

    Args:
        gen (nn.Module) : Generative model under which to calculate the 
                          probabilities for all possible POVM outcomes
    Returns:
        (np.ndarray) : Shape (Na**Nq, 1) with prob. for each possible possible
                       tensor product POVM outcome
    """
    # No. of qubits and no. of POVM elements
    Nq, Na = 3, 6
    outcomes, probs = np.arange(Na**Nq), np.ones(Na**Nq)
    n_batches = math.ceil(len(outcomes)/batch_size)

    # Trying to vectorize int2basestr
    v_int2basestr = np.vectorize(int2basestr, otypes=[np.ndarray])
    batch_sequences = np.zeros((batch_size, Nq))

    for i in range(n_batches):
        if i == n_batches-1:
            batch_outcomes = outcomes[i*batch_size:]
        else:
            batch_outcomes = outcomes[i*batch_size:(i+1)*batch_size]
            
        ret = v_int2basestr(batch_outcomes, Na, l=Nq)
        print(ret)
        for j in range(batch_size):
            batch_sequences[j] = ret[j]
            
        if i == n_batches-1:
            probs[i*batch_size:] = batch_sequences[:, 0]
        else:
            probs[i*batch_size:(i+1)*batch_size] = batch_sequences[:, 0]

    # Vector of dimension (Na**Nq x 1) - main place that could be taking too
    # Note that the samples are not being batched - batch_size = 1
    return probs

In [ ]:
GenPOVMProbTable(10)

[list([0, 0, 0]) list([0, 0, 1]) list([0, 0, 2]) list([0, 0, 3])
 list([0, 0, 4]) list([0, 0, 5]) list([0, 1, 0]) list([0, 1, 1])
 list([0, 1, 2]) list([0, 1, 3])]
[list([0, 1, 4]) list([0, 1, 5]) list([0, 2, 0]) list([0, 2, 1])
 list([0, 2, 2]) list([0, 2, 3]) list([0, 2, 4]) list([0, 2, 5])
 list([0, 3, 0]) list([0, 3, 1])]
[list([0, 3, 2]) list([0, 3, 3]) list([0, 3, 4]) list([0, 3, 5])
 list([0, 4, 0]) list([0, 4, 1]) list([0, 4, 2]) list([0, 4, 3])
 list([0, 4, 4]) list([0, 4, 5])]
[list([0, 5, 0]) list([0, 5, 1]) list([0, 5, 2]) list([0, 5, 3])
 list([0, 5, 4]) list([0, 5, 5]) list([1, 0, 0]) list([1, 0, 1])
 list([1, 0, 2]) list([1, 0, 3])]
[list([1, 0, 4]) list([1, 0, 5]) list([1, 1, 0]) list([1, 1, 1])
 list([1, 1, 2]) list([1, 1, 3]) list([1, 1, 4]) list([1, 1, 5])
 list([1, 2, 0]) list([1, 2, 1])]
[list([1, 2, 2]) list([1, 2, 3]) list([1, 2, 4]) list([1, 2, 5])
 list([1, 3, 0]) list([1, 3, 1]) list([1, 3, 2]) list([1, 3, 3])
 list([1, 3, 4]) list([1, 3, 5])]
[list([1, 4, 0])

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
a = np.ones(10)
a[9:12] = np.zeros(3)

ValueError: could not broadcast input array from shape (3,) into shape (1,)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def GenPOVMProbTable(batch_size):
    """Build POVM probability table using the generator

    Args:
        gen (nn.Module) : Generative model under which to calculate the 
                          probabilities for all possible POVM outcomes
    Returns:
        (np.ndarray) : Shape (Na**Nq, 1) with prob. for each possible possible
                       tensor product POVM outcome
    """
    # No. of qubits and no. of POVM elements
    Nq, Na = 4, 4
    outcomes, probs = np.arange(Na**Nq), np.ones(Na**Nq)
    n_batches = math.ceil(len(outcomes)/batch_size)

    # Trying to vectorize int2basestr
    v_int2basestr = np.vectorize(int2basestr, otypes=[np.ndarray])
    sequences = np.zeros((len(outcomes), Nq))
    
    # Construct sequences
    ret = v_int2basestr(outcomes, Na, l=Nq)
    for j in range(batch_size):
        sequences[j] = ret[j]
        
    seq_data = DataLoader(sequences, batch_size=batch_size)

    for i, seq_batch in enumerate(seq_data):
        if i == n_batches-1:
            probs[i*batch_size:] = (seq_batch[:, 3]).detach().cpu().numpy()
        else:
            probs[i*batch_size:(i+1)*batch_size] = (seq_batch[:, 3]).detach().cpu().numpy()

    # Vector of dimension (Na**Nq x 1) - main place that could be taking too
    # Note that the samples are not being batched - batch_size = 1
    return probs

In [ ]:
outcomes = torch.zeros(4, 6)
outcomes = torch.nn.ConstantPad1d(1, 1)(outcomes)
outcomes[:, -1] = outcomes[:, -1]*2.
outcomes

tensor([[1., 0., 0., 0., 0., 0., 0., 2.],
        [1., 0., 0., 0., 0., 0., 0., 2.],
        [1., 0., 0., 0., 0., 0., 0., 2.],
        [1., 0., 0., 0., 0., 0., 0., 2.]])

In [ ]:
torch.constant_pad_nd

In [ ]:
outcomes += 3
outcomes.size(0)

4

In [ ]:
trg = torch.cat((start, outcomes, end), dim=-1)
trg

RuntimeError: Tensors must have same number of dimensions: got 1 and 2

In [ ]:
ones = torch.ones(4).to(torch.device('cuda:0'))
ones

tensor([1., 1., 1., 1.], device='cuda:0')

In [ ]:
ones = ones.int()
ones

tensor([1, 1, 1, 1], device='cuda:0', dtype=torch.int32)

In [ ]:
torch.cuda.is_available()

True